In [1]:
import os
from ccdc import io
from ccdc import conformer
from ccdc.descriptors import MolecularDescriptors
from raspa_molecule import read_mol2, write_raspa_molecule

In [ ]:
mol_path = 'DMOG.mol2'
mol_name = os.path.splitext(os.path.basename(mol_path))[0]
mol_reader = io.MoleculeReader(mol_path)
mol = mol_reader[0]
mol_reader.close()

In [ ]:
conformer_generator = conformer.ConformerGenerator()   # Initialize conformer generator
conformer_generator.settings.max_conformers = 50       # Set max number of conformers
conformers = conformer_generator.generate(mol)         # Run conformer generator

print("""
Conformers generated: %i
Conformer sampling limit reached: %s
Rotamers sampled: %i / %i
""" % (len(conformers), conformers.sampling_limit_reached, conformers.n_rotamers_sampled, conformers.n_rotamers_in_molecule))

In [ ]:
mol2_conf_dir = '%s-conformers-mol2' % mol_name
raspa_conf_dir = '%s-conformers-raspa' % mol_name

if not os.path.isdir(mol2_conf_dir):
    os.makedirs(mol2_conf_dir)
    
if not os.path.isdir(raspa_conf_dir):
    os.makedirs(raspa_conf_dir)

molecule_minimiser = conformer.MoleculeMinimiser()                   # Uses Tripos force field
min_conformers = []
for conf_idx, conf in enumerate(conformers):
    score, rmsd = conf.normalised_score, conf.rmsd()                 # Conformer score and RMSD
    min_conf = molecule_minimiser.minimise(conf.molecule)            # Minimise conformer
    min_conformers.append(min_conf)                                  # Add to list
    min_rmsd = round( MolecularDescriptors.rmsd(mol, min_conf), 3)   # Minimized RMSD (!!this method gives different results compared to conf.rmsd())
    
    conf_path = os.path.join(mol2_conf_dir, '%s-%i.mol2' % (mol_name, conf_idx + 1))
    with io.MoleculeWriter(conf_path) as molecule_writer:
        molecule_writer.write(min_conf)
        
    mol2_conf = read_mol2(conf_path)
    write_raspa_molecule(mol2_conf, save=raspa_conf_dir, name='%s-%i' % (mol_name, conf_idx + 1))